# Part 1. Install Packages

In [ ]:
%pip install swig wrds pyportfolioopt pandas_market_calendars

In [3]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

import itertools

# Part 2. Fetch data

[yfinance](https://github.com/ranaroussi/yfinance) is an open-source library that provides APIs fetching historical data form Yahoo Finance. In FinRL, we have a class called [YahooDownloader](https://github.com/AI4Finance-Foundation/FinRL/blob/master/finrl/meta/preprocessor/yahoodownloader.py) that use yfinance to fetch data from Yahoo Finance.

**OHLCV**: Data downloaded are in the form of OHLCV, corresponding to **open, high, low, close, volume,** respectively. OHLCV is important because they contain most of numerical information of a stock in time series. From OHLCV, traders can get further judgement and prediction like the momentum, people's interest, market trends, etc.

## Data for a single ticker

Here we provide two ways to fetch data with single ticker, let's take Apple Inc. (AAPL) as an example.

### Using yfinance

In [ ]:
aapl_df_yf = yf.download(tickers = "appl", start='2020-01-01', end='2020-01-31')

[*********************100%***********************]  1 of 1 completed


In [29]:
aapl_df_yf.head()

Price,Close,High,Low,Open,Volume
Ticker,DOW,DOW,DOW,DOW,DOW
Date,,,,,
2020-01-02,40.613365,41.965885,40.530249,41.731649,5145200
2020-01-03,39.608425,40.500031,39.600870,40.129789,5004300
2020-01-06,39.449749,39.956001,39.291074,39.532865,4293100
2020-01-07,39.434635,40.095783,39.283518,39.510197,5122800
2020-01-08,39.842659,40.175121,39.230624,39.600868,3629700


### Using FinRL

In FinRL's YahooDownloader, we modified the data frame to the form that convenient for further data processing process. We use adjusted close price instead of close price, and add a column representing the day of a week (0-4 corresponding to Monday-Friday).

In [22]:
aapl_df_finrl = YahooDownloader(start_date = '2020-01-01',
                                end_date = '2020-01-31',
                                ticker_list = ['aapl']).fetch_data()

[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (20, 8)


In [23]:
aapl_df_finrl.head()

Price,date,close,high,low,open,volume,tic,day
0,2020-01-02,72.716080,75.150002,73.797501,74.059998,135480400,aapl,3
1,2020-01-03,72.009125,75.144997,74.125000,74.287498,146322800,aapl,4
2,2020-01-06,72.582901,74.989998,73.187500,73.447502,118387200,aapl,0
3,2020-01-07,72.241539,75.224998,74.370003,74.959999,108872000,aapl,1
4,2020-01-08,73.403633,76.110001,74.290001,74.290001,132079200,aapl,2


## Data for the chosen tickers

In [30]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2020-07-01'
TRADE_START_DATE = '2020-07-01'
TRADE_END_DATE = '2021-10-29'
TICKERS = [x for x in config_tickers.DOW_30_TICKER if x != 'DOW']
print(TICKERS)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS']


In [31]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = TICKERS).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (93641, 8)


In [32]:
df_raw.head()

Price,date,close,high,low,open,volume,tic,day
0,2009-01-02,2.730994,3.251429,3.041429,3.067143,746015200,AAPL,4
1,2009-01-02,41.151943,59.080002,57.750000,58.590000,6547900,AMGN,4
2,2009-01-02,14.929292,19.520000,18.400000,18.570000,10955700,AXP,4
3,2009-01-02,33.941097,45.560001,42.779999,42.799999,7010200,BA,4
4,2009-01-02,30.490791,46.980000,44.709999,44.910000,7117200,CAT,4


# Part 3: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

Hear let's take **MACD** as an example. Moving average convergence/divergence (MACD) is one of the most commonly used indicator showing bull and bear market. Its calculation is based on EMA (Exponential Moving Average indicator, measuring trend direction over a period of time.)

In [33]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_raw)

[*********************100%***********************]  1 of 1 completed

Successfully added technical indicators
Shape of DataFrame:  (3228, 8)
Successfully added vix


Successfully added turbulence index


In [34]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [35]:
processed_full.head()

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,AAPL,2.730994,3.251429,3.041429,3.067143,746015200.0,4.0,0.0,2.951623,2.625622,100.0,66.666667,100.0,2.730994,2.730994,39.189999,0.0
1,2009-01-02,AMGN,41.151943,59.080002,57.750000,58.590000,6547900.0,4.0,0.0,2.951623,2.625622,100.0,66.666667,100.0,41.151943,41.151943,39.189999,0.0
2,2009-01-02,AXP,14.929292,19.520000,18.400000,18.570000,10955700.0,4.0,0.0,2.951623,2.625622,100.0,66.666667,100.0,14.929292,14.929292,39.189999,0.0
3,2009-01-02,BA,33.941097,45.560001,42.779999,42.799999,7010200.0,4.0,0.0,2.951623,2.625622,100.0,66.666667,100.0,33.941097,33.941097,39.189999,0.0
4,2009-01-02,CAT,30.490791,46.980000,44.709999,44.910000,7117200.0,4.0,0.0,2.951623,2.625622,100.0,66.666667,100.0,30.490791,30.490791,39.189999,0.0


# Part 4: Save the Data

### Split the data for training and trading

In [36]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

83897
9715


### Save data to csv file

For Colab users, you can open the virtual directory in colab and manually download the files.

For users running on your local environment, the csv files should be at the same directory of this notebook.

In [37]:
train.to_csv('train_data.csv')
trade.to_csv('trade_data.csv')